In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv


In [2]:
df=pd.read_csv('/kaggle/input/anemia-types-classification/diagnosed_cbc_data_v4.csv')
df.head()

,WBC,LYMp,NEUTp,LYMn,NEUTn,RBC,HGB,HCT,MCV,MCH,MCHC,PLT,PDW,PCT,Diagnosis
0,10.0,43.2,50.1,4.3,5.0,2.77,7.3,24.2,87.7,26.3,30.1,189.0,12.5,0.17,Normocytic hypochromic anemia
1,10.0,42.4,52.3,4.2,5.3,2.84,7.3,25.0,88.2,25.7,20.2,180.0,12.5,0.16,Normocytic hypochromic anemia
2,7.2,30.7,60.7,2.2,4.4,3.97,9.0,30.5,77.0,22.6,29.5,148.0,14.3,0.14,Iron deficiency anemia
3,6.0,30.2,63.5,1.8,3.8,4.22,3.8,32.8,77.9,23.2,29.8,143.0,11.3,0.12,Iron deficiency anemia
4,4.2,39.1,53.7,1.6,2.3,3.93,0.4,316.0,80.6,23.9,29.7,236.0,12.8,0.22,Normocytic hypochromic anemia


In [3]:
y=df['Diagnosis']#Her zaman 'y' tahmin etmek istediklerimizi simgeler.
#df.head() yaptığımızda tabloda 'Diagnosis' sütununu görmüştük. Oradaki sütunu y'ye atıyoruz.

In [4]:
x=df.drop('Diagnosis',axis=1)#Diagnosis'i atıyor ve geriye kalan hepsini x'e atıyoruz.
#Her zaman 'x' değeri özellikler (features) gösterir.
#axis=1 ekledim çünkü bu sütun bazında işlem yapılacağını belirtir. Bu olmadan hata verir.

In [5]:
from sklearn.preprocessing import LabelEncoder
#LabelEncoder is used to convert categorical labels (like strings) into numeric values.
le=LabelEncoder()#Creates an instance of the LabelEncoder class.
y=le.fit_transform(y)#Learns the unique categories in y and converts those categories into numbers.

In [6]:
le.inverse_transform([4])#Reverse label encoding — converts numeric label back to its original categorical form.

array(['Macrocytic anemia'], dtype=object)

In [7]:
from tensorflow.keras.utils import to_categorical
#Sayısal etiketleri one-hot encoding formatına çevirir. Bu format, özellikle Keras ile çok sınıflı sınıflandırma yaparken softmax aktivasyonlu çıkış katmanları için gereklidir.
y=to_categorical(y)
#convert your target labels y into one-hot encoded format, which is essential for multi-class classification tasks using neural networks like your Keras Sequential model.

2025-08-03 20:19:34.920207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754252375.117541      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754252375.167649      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
y[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0.])

In [9]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.75,random_state=43)

In [10]:
from tensorflow.keras.models import Sequential
model=Sequential()

In [11]:
x.columns #Sütunların isimlerini bir liste olarak gösterir.

Index(['WBC', 'LYMp', 'NEUTp', 'LYMn', 'NEUTn', 'RBC', 'HGB', 'HCT', 'MCV',
       'MCH', 'MCHC', 'PLT', 'PDW', 'PCT'],
      dtype='object')

In [12]:
len(x.columns) #Sütun sayısını gösterir.

14

In [13]:
from tensorflow.keras.layers import Dense

model.add(Dense(64,activation='relu',input_dim=len(x.columns))) #Adding the first hidden layer to neural network.

#Dense(64)=	Adds a fully connected layer with 64 neurons.

#activation='relu'=	Uses ReLU activation to introduce non-linearity.

#input_dim=len(x.columns)=	Sets the input size to match the number of features in x

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1754252386.716495      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [14]:
model.add(Dense(32, activation='relu')) #Adds a hidden layer with 32 neurons and ReLU activation.
model.add(Dense(32, activation='relu')) #This layer learns intermediate features from the previous layer.
model.add(Dense(9, activation='softmax'))#Output layer with 9 neurons (for 9 anemia types).

#'softmax' activation converts outputs into probabilities summing to 1.

In [15]:
help(model.compile) #I need to check this first.

Help on method compile in module keras.src.trainers.trainer:

compile(optimizer='rmsprop', loss=None, loss_weights=None, metrics=None, weighted_metrics=None, run_eagerly=False, steps_per_execution=1, jit_compile='auto', auto_scale_loss=True) method of keras.src.models.sequential.Sequential instance
    Configures the model for training.
    
    Example:
    
    ```python
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            keras.metrics.BinaryAccuracy(),
            keras.metrics.FalseNegatives(),
        ],
    )
    ```
    
    Args:
        optimizer: String (name of optimizer) or optimizer instance. See
            `keras.optimizers`.
        loss: Loss function. May be a string (name of loss function), or
            a `keras.losses.Loss` instance. See `keras.losses`. A
            loss function is any callable with the signature
            `loss = fn(y_true, y_pred)`, wher

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
#oAdam optimizer adapts learning rates during training for faster convergence. It’s a great default choice.

#Specifies the loss function for multi-class classification when your labels are one-hot encoded.

In [17]:
model.fit(x_train,y_train,epochs=128,batch_size=32,validation_split=0.24) 

#.fit() is training Keras model on the anemia dataset.

Epoch 1/128


I0000 00:00:1754252389.928761      59 service.cc:148] XLA service 0x781f0c00b800 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754252389.929316      59 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1754252390.167012      59 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/23 ━━━━━━━━━━━━━━━━━━━━ 1:00 3s/step - accuracy: 0.0000e+00 - loss: 46.4003

I0000 00:00:1754252390.882609      59 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23/23 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.1099 - loss: 23.1939 - val_accuracy: 0.2511 - val_loss: 5.0063
Epoch 2/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2690 - loss: 3.9403 - val_accuracy: 0.3160 - val_loss: 2.6948
Epoch 3/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4043 - loss: 2.0520 - val_accuracy: 0.3853 - val_loss: 2.0639
Epoch 4/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3816 - loss: 1.8610 - val_accuracy: 0.4848 - val_loss: 1.9154
Epoch 5/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4379 - loss: 1.8368 - val_accuracy: 0.4632 - val_loss: 1.7759
Epoch 6/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4564 - loss: 1.5522 - val_accuracy: 0.4935 - val_loss: 1.6635
Epoch 7/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4625 - loss: 1.5726 - val_accuracy: 0.5325 - val_loss: 1.6974
Epoch 8/128
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4791 - loss: 1.5858 - val_accuracy: 0.4675 - val_loss: 1